# **Land Surface Temperature of Dubai from 1995 and 2024**

# Urban Evolution of Dubai - A Comparative Remote Sensing Analysis from 1995 to 2024

**Erik Ashkinadze (erik.ashkinadze@ruhr-uni-bochum.de)**

**Devon Klör (devon.kloer@ruhr-uni-bochum.de)**

**Course:** Geographic Information Systems (GIS I): Databases and Programming

**Professor:** Jun.-Prof. Dr. Andreas Rienow

**References:** Ridho, M. (2023): Analyzing Land Surface Temperature (LST) with Landsat 8 Data in Google Earth Engine.
https://medium.com/@ridhomuh002/analyzing-land-surface-temperature-lst-with-landsat-8-data-in-google-earth-engine-f4dd7ca28e70

In [1]:
import ee
import geemap

# Authenticate and Initialize Earth Engine with your project ID
ee.Authenticate()
ee.Initialize(project='ee-dkloer01')

# import Dubai Shapefile
shapefile_path = "./Dubai_Shapes/Dubai.shp"
dubai = geemap.shp_to_ee(shapefile_path)

# Landsat Data Collections with Date Cloud Filter
def get_landsat_data(collection, start_date, end_date, dubai):
    return (ee.ImageCollection(collection)
            .filterBounds(dubai)
            .filterDate(start_date, end_date)
            .filterMetadata("CLOUD_COVER", "less_than", 20)
            .median())

# Load tghe Landsat 5 and 8 dataset for historical and current analysis
dubai_1995 = get_landsat_data('LANDSAT/LT05/C02/T1_TOA', '1995-01-01', '1995-12-31', dubai)
dubai_2024 = get_landsat_data('LANDSAT/LC08/C02/T1_TOA', '2024-01-01', '2024-12-31', dubai)

# Calculate the NDVI
ndvi_1995 = dubai_1995.normalizedDifference(['B4', 'B3']).rename('NDVI')
ndvi_2024 = dubai_2024.normalizedDifference(['B5', 'B4']).rename('NDVI')

# Calculate Fractional Vegetation Cover (FV)
ndvi_min = ee.Number(-1)
ndvi_max = ee.Number(1)
fv_1995 = ((ndvi_1995.subtract(ndvi_min)).divide(ndvi_max.subtract(ndvi_min))).pow(2).rename('FV')
fv_2024 = ((ndvi_2024.subtract(ndvi_min)).divide(ndvi_max.subtract(ndvi_min))).pow(2).rename('FV')

# Calculate Emissivity
em_1995 = fv_1995.multiply(0.004).add(0.986).rename('EM')
em_2024 = fv_2024.multiply(0.004).add(0.986).rename('EM')

# LST Calculation Function
def calculate_LST(thermal, em):
    return thermal.expression(
        '(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15', {
            'TB': thermal,
            'em': em
        }).rename('LST')


# Calculate LST for each year
lst_1995 = calculate_LST(dubai_1995.select('B6'), em_1995)  # Landsat 5 Band 6
lst_2024 = calculate_LST(dubai_2024.select('B11'), em_2024)  # Landsat 8 Band 11

# Parameter for LST visualization
lst_vis = {
    'min': 10, 'max': 40,
    'palette': [
        '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
        '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
        '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
        'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
        'ff0000', 'de0101', 'c21301', 'a71001', '911003'
    ]
}

# Create an interactive Map with Geemap
map = geemap.Map(center=[25.07, 55.18], zoom=10)

# Add the Study Area as a Maplayer
map.addLayer(dubai.style(**{'color': 'red', 'width': 2}), {}, 'Study Area')

# Add both LST as a Maplayer
map.addLayer(lst_1995.clip(dubai), lst_vis, 'LST 1995')
map.addLayer(lst_2024.clip(dubai), lst_vis, 'LST 2024')

# Show the Map
map


Map(center=[25.07, 55.18], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…